In [ ]:
import pandas as pd
import preprocess as pre
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Phrases
# Remove rare and common tokens.
from gensim.corpora import Dictionary
# Train LDA model.
from gensim.models import LdaModel
from pprint import pprint
import numpy as np
import logging
from gensim.models import CoherenceModel
from preprocess import processbody


In [ ]:
dataset = "datadog"
df = pd.read_csv('../dataset/{}-aiops.csv'.format(dataset))
docs = []
df['clean_text'] = ''
for index, row in df.iterrows():
    title = processbody(row['title'])
    body = processbody(row['body'])
    s = (title+" "+body).strip()
    df.at[index, 'clean_text'] = docs.append(s)

print(len(docs))
print(docs[0][:50])

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]
docs = [[token for token in doc if len(token) <= 15] for doc in docs]
np.save('{}_docs.npy'.format(dataset), np.array(docs))
# docs = [doc for doc in docs if len(doc)>1]

In [ ]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
dictionary.filter_extremes(no_below=2)
dictionary.save('{}.dict'.format(dataset))

corpus = [dictionary.doc2bow(doc) for doc in docs]
np.save('{}_corpus.npy'.format(dataset),np.array(corpus))
# corpus = np.load('./data/dl2.npy', allow_pickle=True).tolist()

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

In [ ]:
df = pd.read_csv("elastic-aiops.csv")

phase0 = df[df['phase']==0]
phase1 = df[df['phase']==1]
phase2 = df[df['phase']==2]
phase3 = df[df['phase']==3]
p01 = phase0[phase0['category'] == 'Installation']
p02 = phase0[phase0['category'] == 'Third-party']
p03 = phase0[phase0['category'] == 'Configuration']
p11 = phase1[phase1['category'] == 'Method']
p12 = phase1[phase1['category'] == 'Error']
p13 = phase1[phase1['category'] == 'Parse']
p21 = phase2[phase2['category'] == 'Storage']
p22 = phase2[phase2['category'] == 'Query']
p23 = phase2[phase2['category'] == 'Error']
p31 = phase3[phase3['category'] == 'Visualization']
p32 = phase3[phase3['category'] == 'Monitoring']

In [ ]:
phase01_popularity = p01[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
phase01col_mean = phase01_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

phase02_popularity = p02[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
phase02col_mean = phase02_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

phase03_popularity = p03[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
phase03col_mean = phase03_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

phase11_popularity = p11[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
phase11col_mean = phase11_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

phase12_popularity = p12[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
phase12col_mean = phase12_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

phase13_popularity = p13[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
phase13col_mean = phase13_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

phase21_popularity = p21[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
phase21col_mean = phase21_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

phase22_popularity = p22[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
phase22col_mean = phase22_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

phase23_popularity = p23[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
phase23col_mean = phase23_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

phase31_popularity = p31[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
phase31col_mean = phase31_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

phase32_popularity = p32[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
phase32col_mean = phase32_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()


In [ ]:
popularity = pd.concat([phase01col_mean, phase02col_mean, phase03col_mean, phase11col_mean, phase12col_mean, phase13col_mean, phase21col_mean, phase22col_mean, phase23col_mean, phase31col_mean, phase32col_mean], axis=1)
#统计受欢迎程度
# popularity.to_csv("{}-popularity.csv".format("elastic"))
# print(popularity)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import mcdm
scaler = MinMaxScaler()
scaler.fit(popularity.T)
scaled_features = scaler.transform(popularity.T)
alt_names = ["01", "02", "03", "11", "12", "13", "21", "22", "23", "31", "32"]
df_MinMax = pd.DataFrame(data=scaled_features, columns=["view_count", "answer_count", "comment_count","favorite_count", "score"])
popularityrank = mcdm.rank(df_MinMax.values,alt_names=alt_names, n_method="Linear2", w_method="CRITIC", s_method="TOPSIS")
print(popularityrank)

In [ ]:
difficulty1 = popularity.iloc[1]/popularity.iloc[0]
difficulty1

In [ ]:
import numpy as np
cnt01 = 0
for index, row in p01.iterrows():
    if np.isnan(row['accepted_answer_id']):
        cnt01 += 1

cnt02 = 0
for index, row in p02.iterrows():
    if np.isnan(row['accepted_answer_id']):
        cnt02 += 1

cnt03 = 0
for index, row in p03.iterrows():
    if np.isnan(row['accepted_answer_id']):
        cnt03 += 1

cnt11 = 0
for index, row in p11.iterrows():
    if np.isnan(row['accepted_answer_id']):
        cnt11 += 1

cnt12 = 0
for index, row in p12.iterrows():
    if np.isnan(row['accepted_answer_id']):
        cnt12 += 1

cnt13 = 0
for index, row in p13.iterrows():
    if np.isnan(row['accepted_answer_id']):
        cnt13 += 1

cnt21 = 0
for index, row in p21.iterrows():
    if np.isnan(row['accepted_answer_id']):
        cnt21 += 1

cnt22 = 0
for index, row in p22.iterrows():
    if np.isnan(row['accepted_answer_id']):
        cnt22 += 1

cnt23 = 0
for index, row in p23.iterrows():
    if np.isnan(row['accepted_answer_id']):
        cnt23 += 1

cnt31 = 0
for index, row in p31.iterrows():
    if np.isnan(row['accepted_answer_id']):
        cnt31 += 1

cnt32 = 0
for index, row in p32.iterrows():
    if np.isnan(row['accepted_answer_id']):
        cnt32 += 1


In [ ]:
unanswered = np.array([cnt01, cnt02, cnt03, cnt11, cnt12, cnt13, cnt21, cnt22, cnt23, cnt31, cnt32], dtype=float)
sum = np.array([len(p01), len(p02), len(p03), len(p11), len(p12), len(p13), len(p21), len(p22), len(p23), len(p31), len(p32)], dtype=float)
w_o = unanswered/sum
print(w_o)

In [ ]:
data = [difficulty1.values, w_o]
difficulty = pd.DataFrame(data=data, index=['pd', 'w/o'], columns=["01", "02", "03", "11", "12", "13", "21", "22", "23", "31", "32"])
difficulty[["01", "02", "03", "11", "12", "13", "21", "22", "23", "31", "32"]]

In [ ]:
import time
import pymysql
#连接数据库
def connectDB():
    # 打开数据库连接
    db = pymysql.connect(host="localhost", user="root", password="Wangnima258", database="aiops")
    print("连接上了...")
    return db
#查询操作
def queryDb(db, sql):
    # 使用 cursor() 方法创建一个游标对象 cursor
    cursor = db.cursor()
    cursor.execute(sql)
    # 获取所有记录列表
    results = cursor.fetchone()
    return results

db = connectDB()
cnt01 = 0
span01 = 0
for index, row in p01.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span01 += span
        cnt01 += 1
hours01 = span01/cnt01

cnt02 = 0
span02 = 0
for index, row in p02.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span02 += span
        cnt02 += 1
hours02 = span02/cnt02

cnt03 = 0
span03 = 0
for index, row in p03.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span03 += span
        cnt03 += 1
hours03 = span03/cnt03

cnt11 = 0
span11 = 0
for index, row in p11.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span11 += span
        cnt11 += 1
hours11 = span11/cnt11

cnt12 = 0
span12 = 0
for index, row in p12.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span12 += span
        cnt12 += 1
hours12 = span12/cnt12

cnt13 = 0
span13 = 0
for index, row in p13.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span13 += span
        cnt13 += 1
hours13 = span13/cnt13

cnt21 = 0
span21 = 0
for index, row in p21.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span21 += span
        cnt21 += 1
hours21 = span21/cnt21

cnt22 = 0
span22 = 0
for index, row in p22.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span22 += span
        cnt22 += 1
hours22 = span22/cnt22

cnt23 = 0
span23 = 0
for index, row in p23.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span23 += span
        cnt23 += 1
hours23 = span23/cnt23

cnt31 = 0
span31 = 0
for index, row in p31.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span31 += span
        cnt31 += 1
hours31 = span31/cnt31

cnt32 = 0
span32 = 0
for index, row in p32.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span32 += span
        cnt32 += 1
hours32 = span32/cnt32

db.close()

In [ ]:
hours = np.array([hours01, hours02, hours03, hours11, hours12, hours13, hours21, hours22, hours23, hours31, hours32], dtype=float)
data = [difficulty1.values, w_o, hours]
difficulty = pd.DataFrame(data=data, index=['pd', 'w/o', 'days'], columns=["0", "1", "2", "3"])
# difficulty.to_csv("elastic-difficulty.csv")

In [ ]:
scaler.fit(difficulty.T)
scaled_features = scaler.transform(difficulty.T)
alt_names =["01", "02", "03", "11", "12", "13", "21", "22", "23", "31", "32"]
df_MinMax = pd.DataFrame(data=scaled_features, columns=['pd', 'w/o', 'days'])
difficultyrank = mcdm.rank(df_MinMax.values, alt_names=alt_names,n_method="Linear2", w_method="CRITIC", s_method="TOPSIS")
difficultyrank

In [ ]:
sdf = pd.read_csv("../dataset/splunk-aiops.csv")
sphase0 = sdf[sdf['phase']==0]
sphase1 = sdf[sdf['phase']==1]
sphase2 = sdf[sdf['phase']==2]
sphase3 = sdf[sdf['phase']==3]
sp01 = sphase0[sphase0['category'] == 'Installation']
sp02 = sphase0[sphase0['category'] == 'Configuration']
sp11 = sphase1[sphase1['category'] == 'Method']
sp12 = sphase1[sphase1['category'] == 'Error']
sp13 = sphase1[sphase1['category'] == 'Parse']
sp21 = sphase2[sphase2['category'] == 'Query']
sp22 = sphase2[sphase2['category'] == 'Error']
sp31 = sphase3[sphase3['category'] == 'Visualization']
sp32 = sphase3[sphase3['category'] == 'Monitoring']

In [ ]:
sphase01_popularity = sp01[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
sphase01col_mean = sphase01_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

sphase02_popularity = sp02[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
sphase02col_mean = sphase02_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

sphase11_popularity = sp11[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
sphase11col_mean = sphase11_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

sphase12_popularity = sp12[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
sphase12col_mean = sphase12_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

sphase13_popularity = sp13[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
sphase13col_mean = sphase13_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

sphase21_popularity = sp21[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
sphase21col_mean = sphase21_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

sphase22_popularity = sp22[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
sphase22col_mean = sphase22_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

sphase31_popularity = sp31[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
sphase31col_mean = sphase31_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

sphase32_popularity = sp32[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
sphase32col_mean = sphase32_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

In [ ]:
spopularity = pd.concat([sphase01col_mean, sphase02col_mean, sphase11col_mean, spase12col_mean, sphase13col_mean, sphase21col_mean, sphase22col_mean, sphase31col_mean, sphase32col_mean], axis=1)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import mcdm
scaler = MinMaxScaler()
scaler.fit(spopularity.T)
scaled_features = scaler.transform(spopularity.T)
alt_names = ["01", "02", "11", "12", "13", "21", "22", "31", "32"]
df_MinMax = pd.DataFrame(data=scaled_features, columns=["view_count", "answer_count", "comment_count","favorite_count", "score"])
spopularityrank = mcdm.rank(df_MinMax.values, alt_names=alt_names,n_method="Linear2", w_method="CRITIC", s_method="TOPSIS")
spopularityrank

In [ ]:
sdifficulty1 = spopularity.iloc[1]/spopularity.iloc[0]
sdifficulty1

In [ ]:
import numpy as np
scnt01 = 0
for index, row in sp01.iterrows():
    if np.isnan(row['accepted_answer_id']):
        scnt01 += 1

scnt02 = 0
for index, row in sp02.iterrows():
    if np.isnan(row['accepted_answer_id']):
        scnt02 += 1

scnt11 = 0
for index, row in sp11.iterrows():
    if np.isnan(row['accepted_answer_id']):
        scnt11 += 1

scnt12 = 0
for index, row in sp12.iterrows():
    if np.isnan(row['accepted_answer_id']):
        scnt12 += 1

scnt13 = 0
for index, row in sp13.iterrows():
    if np.isnan(row['accepted_answer_id']):
        scnt13 += 1

scnt21 = 0
for index, row in sp21.iterrows():
    if np.isnan(row['accepted_answer_id']):
        scnt21 += 1

scnt22 = 0
for index, row in sp22.iterrows():
    if np.isnan(row['accepted_answer_id']):
        scnt22 += 1

scnt31 = 0
for index, row in sp31.iterrows():
    if np.isnan(row['accepted_answer_id']):
        scnt31 += 1

scnt32 = 0
for index, row in sp32.iterrows():
    if np.isnan(row['accepted_answer_id']):
        scnt32 += 1


In [ ]:
unanswered = np.array([scnt01, scnt02, scnt11, scnt12, scnt13, scnt21, scnt22, scnt31, scnt32], dtype=float)
sum = np.array([len(sp01), len(sp02), len(sp11), len(sp12), len(sp13), len(sp21), len(sp22), len(sp31), len(sp32)], dtype=float)
w_o = unanswered/sum
print(w_o)

In [ ]:
sdata = [sdifficulty1.values, w_o]
sdifficulty = pd.DataFrame(data=data, index=['pd', 'w/o'], columns=["01", "02", "11", "12", "13", "21", "22", "31", "32"])
sdifficulty[["01", "02", "11", "12", "13", "21", "22", "31", "32"]]

In [ ]:
import time
import pymysql
#连接数据库
def connectDB():
    # 打开数据库连接
    db = pymysql.connect(host="localhost", user="root", password="Wangnima258", database="aiops")
    print("连接上了...")
    return db
#查询操作
def queryDb(db, sql):
    # 使用 cursor() 方法创建一个游标对象 cursor
    cursor = db.cursor()
    cursor.execute(sql)
    # 获取所有记录列表
    results = cursor.fetchone()
    return results

db = connectDB()
cnt01 = 0
span01 = 0
for index, row in sp01.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span01 += span
        cnt01 += 1
hours01 = span01/cnt01

cnt02 = 0
span02 = 0
for index, row in sp02.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span02 += span
        cnt02 += 1
hours02 = span02/cnt02

cnt11 = 0
span11 = 0
for index, row in sp11.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span11 += span
        cnt11 += 1
hours11 = span11/cnt11

cnt12 = 0
span12 = 0
for index, row in sp12.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span12 += span
        cnt12 += 1
hours12 = span12/cnt12

cnt13 = 0
span13 = 0
for index, row in sp13.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span13 += span
        cnt13 += 1
hours13 = span13/cnt13

cnt21 = 0
span21 = 0
for index, row in sp21.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span21 += span
        cnt21 += 1
hours21 = span21/cnt21

cnt22 = 0
span22 = 0
for index, row in sp22.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span22 += span
        cnt22 += 1
hours22 = span22/cnt22

cnt31 = 0
span31 = 0
for index, row in sp31.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span31 += span
        cnt31 += 1
hours31 = span31/cnt31

cnt32 = 0
span32 = 0
for index, row in sp32.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span32 += span
        cnt32 += 1
hours32 = span32/cnt32

db.close()

In [ ]:
hours = np.array([hours01, hours02, hours11, hours12, hours13, hours21, hours22, hours31, hours32], dtype=float)
data = [sdifficulty1.values, w_o, hours]
sdifficulty = pd.DataFrame(data=data, index=['pd', 'w/o', 'days'], columns=["01", "02", "11", "12", "13", "21", "22", "31", "32"])
# splunkdifficulty.to_csv("splunk-difficulty.csv")

In [ ]:
scaler.fit(sdifficulty.T)
scaled_features = scaler.transform(sdifficulty.T)
alt_names = ["01", "02", "11", "12", "13", "21", "22", "31", "32"]
df_MinMax = pd.DataFrame(data=scaled_features, columns=['pd', 'w/o', 'days'])
splunkdifficultyrank = mcdm.rank(df_MinMax.values, alt_names=alt_names,n_method="Linear2", w_method="CRITIC", s_method="TOPSIS")
splunkdifficultyrank

In [ ]:
ddf = pd.read_csv("datadog-aiops.csv")
dphase0 = ddf[ddf['phase']==0]
dphase1 = ddf[ddf['phase']==1]
dphase2 = ddf[ddf['phase']==2]
dphase3 = ddf[ddf['phase']==3]
dp01 = dphase0[dphase0['category'] == 'Integration']
dp02 = dphase0[dphase0['category'] == 'Configuration']
dp11 = dphase1[dphase1['category'] == 'Method']
dp12 = dphase1[dphase1['category'] == 'Error']
dp13 = dphase1[dphase1['category'] == 'Parse']
dp21 = dphase2[dphase2['category'] == 'Query']
dp31 = dphase3[dphase3['category'] == 'Visualization']
dp32 = dphase3[dphase3['category'] == 'Monitoring']

In [ ]:
dphase01_popularity = dp01[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
dphase01col_mean = dphase01_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

dphase02_popularity = dp02[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
dphase02col_mean = dphase02_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

dphase11_popularity = dp11[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
dphase11col_mean = dphase11_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

dphase12_popularity = dp12[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
dphase12col_mean = dphase12_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

dphase13_popularity = dp13[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
dphase13col_mean = dphase13_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

dphase21_popularity = dp21[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
dphase21col_mean = dphase21_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

dphase31_popularity = dp31[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
dphase31col_mean = dphase31_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

dphase32_popularity = dp32[['post_id','creation_date','view_count','answer_count','comment_count','favorite_count','score']]
dphase32col_mean = dphase32_popularity[['view_count','answer_count','comment_count','favorite_count','score']].mean()

In [ ]:
dpopularity = pd.concat([dphase01col_mean, dphase02col_mean, dphase11col_mean, dphase12col_mean, dphase13col_mean, dphase21col_mean, dphase31col_mean, dphase32col_mean], axis=1)
#统计受欢迎程度
# datadogpopularity.to_csv("{}-popularity.csv".format("datadog"))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import mcdm
scaler = MinMaxScaler()
scaler.fit(dpopularity.T)
scaled_features = scaler.transform(dpopularity.T)
alt_names = ["01", "02", "11", "12", "13", "21", "31", "32"]
df_MinMax = pd.DataFrame(data=scaled_features, columns=["view_count", "answer_count", "comment_count","favorite_count", "score"])
datadogpopularityrank = mcdm.rank(df_MinMax.values, alt_names=alt_names,n_method="Linear2", w_method="CRITIC", s_method="TOPSIS")
datadogpopularityrank

In [ ]:
ddifficulty1 = dpopularity.iloc[1]/dpopularity.iloc[0]
ddifficulty1

In [ ]:
import numpy as np
dcnt01 = 0
for index, row in dp01.iterrows():
    if np.isnan(row['accepted_answer_id']):
        dcnt01 += 1

dcnt02 = 0
for index, row in dp02.iterrows():
    if np.isnan(row['accepted_answer_id']):
        dcnt02 += 1

dcnt11 = 0
for index, row in dp11.iterrows():
    if np.isnan(row['accepted_answer_id']):
        dcnt11 += 1

dcnt12 = 0
for index, row in dp12.iterrows():
    if np.isnan(row['accepted_answer_id']):
        dcnt12 += 1

dcnt13 = 0
for index, row in dp13.iterrows():
    if np.isnan(row['accepted_answer_id']):
        dcnt13 += 1

dcnt21 = 0
for index, row in dp21.iterrows():
    if np.isnan(row['accepted_answer_id']):
        dcnt21 += 1

dcnt31 = 0
for index, row in dp31.iterrows():
    if np.isnan(row['accepted_answer_id']):
        dcnt31 += 1

dcnt32 = 0
for index, row in dp32.iterrows():
    if np.isnan(row['accepted_answer_id']):
        dcnt32 += 1

In [ ]:
unanswered = np.array([dcnt01, dcnt02, dcnt11, dcnt12, dcnt13, dcnt21, dcnt31, dcnt32], dtype=float)
sum = np.array([len(dp01), len(dp02), len(dp11), len(dp12), len(dp13), len(dp21), len(dp31), len(dp32)], dtype=float)
w_o = unanswered/sum
print(w_o)

In [ ]:
data = [ddifficulty1.values, w_o]
ddifficulty = pd.DataFrame(data=data, index=['pd', 'w/o'], columns=["01", "02", "11", "12", "13", "21", "31", "32"])
ddifficulty[["01", "02", "11", "12", "13", "21", "31", "32"]]

In [ ]:
import time
import pymysql
#连接数据库
def connectDB():
    # 打开数据库连接
    db = pymysql.connect(host="localhost", user="root", password="Wangnima258", database="aiops")
    print("连接上了...")
    return db
#查询操作
def queryDb(db, sql):
    # 使用 cursor() 方法创建一个游标对象 cursor
    cursor = db.cursor()
    cursor.execute(sql)
    # 获取所有记录列表
    results = cursor.fetchone()
    return results
db = connectDB()
cnt01 = 0
span01 = 0
for index, row in sp01.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span01 += span
        cnt01 += 1
hours01 = span01/cnt01

cnt02 = 0
span02 = 0
for index, row in sp02.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span02 += span
        cnt02 += 1
hours02 = span02/cnt02

cnt11 = 0
span11 = 0
for index, row in dp11.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span11 += span
        cnt11 += 1
hours11 = span11/cnt11

cnt12 = 0
span12 = 0
for index, row in dp12.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span12 += span
        cnt12 += 1
hours12 = span12/cnt12

cnt13 = 0
span13 = 0
for index, row in dp13.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span13 += span
        cnt13 += 1
hours13 = span13/cnt13

cnt21 = 0
span21 = 0
for index, row in dp21.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span21 += span
        cnt21 += 1
hours21 = span21/cnt21

cnt31 = 0
span31 = 0
for index, row in dp31.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span31 += span
        cnt31 += 1
hours31 = span31/cnt31

cnt32 = 0
span32 = 0
for index, row in dp32.iterrows():
    if not np.isnan(row['accepted_answer_id']):
        # print(row['CreationDate'])
        start = time.mktime(time.strptime(row['creation_date'], '%Y-%m-%d %H:%M:%S'))
        Aid = str(row['accepted_answer_id'])
        sql = "select creation_date from answers where post_id = " + Aid
        result = queryDb(db, sql)
        if(result is None):
            continue
        # print(answer.iloc[0]['CreationDate'])
        end = time.mktime(time.strptime(str(result[0]), '%Y-%m-%d %H:%M:%S'))
        # days
        span = round(end - start) / (3600*24)
        span32 += span
        cnt32 += 1
hours32 = span32/cnt32

db.close()

In [ ]:
hours = np.array([hours01, hours02, hours11, hours12, hours13, hours21, hours31, hours32], dtype=float)
data = [ddifficulty1.values, w_o, hours]
ddifficulty = pd.DataFrame(data=data, index=['pd', 'w/o', 'days'], columns=["01", "02", "11", "12", "13", "21", "31", "32"])

In [ ]:
scaler.fit(ddifficulty.T)
scaled_features = scaler.transform(ddifficulty.T)
alt_names = ["01", "02", "11", "12", "13", "21", "31", "32"]
df_MinMax = pd.DataFrame(data=scaled_features, columns=['pd', 'w/o', 'days'])
datadogdifficultyrank = mcdm.rank(df_MinMax.values, alt_names=alt_names,n_method="Linear2", w_method="CRITIC", s_method="TOPSIS")
datadogdifficultyrank